## Using REST API for a getting crypto prices and making a Candlestick chart ##

In [3]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance

In [5]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

Using a REST API on the CoinGecko website. Here is the link [CoinGecko API](https://www.coingecko.com/en/api?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01)

Here is [API Documentation] (https://www.coingecko.com/en/api/documentation)

Applying a Python client/wrapper for the API called [Pycoingecko](https://github.com/man-c/pycoingecko?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01%3EPyCoinGecko) 

In [7]:
cg = CoinGeckoAPI()

We will use a *contract* endpoint  
**/coins/{id}/contract/{contract_address}/market_chart/ (Get historical market data include price, market cap, and 24h volume (granularity auto) from a contract address)**

In [11]:
miota_data = cg.get_coin_market_chart_by_id(id='iota', vs_currency='usd', days=30)
miota_data.keys()

dict_keys(['prices', 'market_caps', 'total_volumes'])

In [15]:
miota_prices = miota_data["prices"]
prices = pd.DataFrame(miota_prices, columns = ["Time", "Price"])
prices.head()

,Time,Price
0,1645881267216,0.745175
1,1645884892489,0.748533
2,1645888499099,0.750524
3,1645892072970,0.754905
4,1645895652809,0.749109


we need to covert a time from timestamp to date with a help of fromtimestamp function

In [20]:
prices['Date'] = prices['Time'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))
prices.head()

,Time,Price,Date
0,1645881267216,0.745175,2022-02-26
1,1645884892489,0.748533,2022-02-26
2,1645888499099,0.750524,2022-02-26
3,1645892072970,0.754905,2022-02-26
4,1645895652809,0.749109,2022-02-26


**Preparing data for a graph using groupby function by Date ang using aggregation per column Prices)**

In [21]:
#for prices we will take min, max, first and last prices
candlestick_data = prices.groupby(prices.Date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

In [27]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['Date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False, xaxis_title = "Date", yaxis_title = "Price(USD $)",
                  title = "Miota Candlestick Chart Over Past 30 Days")


plot(fig, filename = "Miota_candlestick_graph.html")
fig.show()